# DB2-Salesforce populate survey results obtain from DB2

In [1]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')
wang159_myrmekes_db = application.new_db_engine('wang159_myrmekes')


salesforce = application.new_salesforce_engine()
db_s = salesforce

In [2]:
import pandas as pd
import datetime

## Get the URL code from DB2


In [3]:
# page_id indicates the question
survey_df = pd.read_sql_query("select user_id,input from jos_reply_replies;",nanohub_db) #limit 5000

In [4]:
display(survey_df.head(2))

,user_id,input
0,170942,"{""text"":""Test.""}"
1,206944,"{""text"":""NCN staff test.""}"


In [5]:
## some processing
s_temp = [i.split('":"')[-1][:-2] for i in survey_df['input'].to_list()]

survey_df['NH_survey_results__c'] = s_temp

In [6]:
display(survey_df.head(2))
display(survey_df.tail(2))

,user_id,input,NH_survey_results__c
0,170942,"{""text"":""Test.""}",Test.
1,206944,"{""text"":""NCN staff test.""}",NCN staff test.


,user_id,input,NH_survey_results__c
83,164228,"{""text"":""This is Tanya on 5\/6\/2021 submittin...",This is Tanya on 5\/6\/2021 submitting somethi...
84,47430,"{""text"":""Testing. This is Lynn\u2019s test ac...",Testing. This is Lynn\u2019s test account. Z...


In [7]:
survey_df = survey_df.drop(columns=['input'])

In [9]:
display(survey_df.head(5))

,user_id,NH_survey_results__c
0,170942,Test.
1,206944,NCN staff test.
2,3482,"This is a test - Gerhard Klimeck, \r\nHenry Bu..."
3,3482,"Well, let's see what happens when I type anoth..."
4,279076,We (the group of Prof. James Glazier at Indian...


In [10]:
survey_df = survey_df.rename(columns={'user_id':'nanoHUB_user_ID__c'})
display(survey_df.head(2))

,nanoHUB_user_ID__c,NH_survey_results__c
0,170942,Test.
1,206944,NCN staff test.


In [11]:
## check for duplicates, if they exist, then merge
duplicates = pd.concat(g for _, g in survey_df.groupby("nanoHUB_user_ID__c") if len(g) > 1)

display(duplicates)

,nanoHUB_user_ID__c,NH_survey_results__c
2,3482,"This is a test - Gerhard Klimeck, \r\nHenry Bu..."
3,3482,"Well, let's see what happens when I type anoth..."
49,164228,Wow- what a lot of questions! I don\u2019t k ...
50,164228,Wow- what a lot of questions! I don\u2019t k ...
51,164228,Did you preview this or get any feedback befor...
83,164228,This is Tanya on 5\/6\/2021 submitting somethi...
1,206944,NCN staff test.
6,206944,Test 4\/23


In [12]:
nh_ids = duplicates['nanoHUB_user_ID__c'].to_list()
nh_results = duplicates['NH_survey_results__c'].to_list()

In [13]:
from collections import Counter

In [14]:
# build new array
dict_nh_ids = Counter(nh_ids)

nh_uniques_ids = [i for i in dict_nh_ids.keys()]
nh_uniques_freq = [i for i in dict_nh_ids.values()]

In [15]:
survey_res_uniques = []
dups_list = duplicates['NH_survey_results__c'].to_list()

counter = 0
for i,j in enumerate(nh_uniques_freq):
    temp_survey_res = []
    for k in range(j):
        temp_survey_res.append(dups_list[counter+k])
        
    survey_res_uniques.append('\n '.join(temp_survey_res) )
    
    counter += j

In [18]:
dups_df = pd.DataFrame()
dups_df['nanoHUB_user_ID__c'] = nh_uniques_ids
dups_df['NH_survey_results__c'] = survey_res_uniques
display(dups_df.head(5))

,nanoHUB_user_ID__c,NH_survey_results__c
0,3482,"This is a test - Gerhard Klimeck, \r\nHenry Bu..."
1,164228,Wow- what a lot of questions! I don\u2019t k ...
2,206944,NCN staff test.\n Test 4\/23


In [ ]:
## send both survey_df and dups_df; dups after survey 

## Send to SF

Obtained Salesforce access token ...... True


In [20]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(survey_df)

[Success] Bulk job creation successful. Job ID = 7505w00000VryNfAAJ
hello
[Success] CSV upload successful. Job ID = 7505w00000VryNfAAJ
[Success] Closing job successful. Job ID = 7505w00000VryNfAAJ


In [28]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000VryNfAAJ',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-05-07T04:43:03.000+0000',
 'systemModstamp': '2021-05-07T04:43:24.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 85,
 'numberRecordsFailed': 8,
 'retries': 0,
 'totalProcessingTime': 2091,
 'apiActiveProcessingTime': 1934,
 'apexProcessingTime': 542}

In [29]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",NH_survey_results__c,nanoHUB_user_ID__c\n'
 '"","DUPLICATE_VALUE:Duplicate external id specified: '
 '206944.0:nanoHUB_user_ID__c --","NCN staff test.","206944.0"\n'
 '"","DUPLICATE_VALUE:Duplicate external id specified: '
 '3482.0:nanoHUB_user_ID__c --","This is a test - Gerhard Klimeck, \\r\\nHenry '
 'Buzz off.\\r\\nI do have a google scholar '
 'page.\\r\\nhttps:\\/\\/scholar.google.com\\/citations?user=t_ziGRgAAAAJ&hl=en\\r\\nand '
 '\\r\\nhttps:\\/\\/www.linkedin.com\\/in\\/gerhardklimeck\\r\\n\\r\\nand\\r\\nhttps:\\/\\/www.researchgate.net\\/profile\\/Gerhard-Klimeck\\r\\n\\r\\nhttps:\\/\\/publons.com\\/researcher\\/2763812\\/gerhard-klimeck\\/\\r\\n\\r\\nhttps:\\/\\/orcid.org\\/0000-0001-7128-773X\\r\\n\\r\\nhttps:\\/\\/www.scopus.com\\/authid\\/detail.uri?authorId=56250324100","3482.0"\n'
 '"","DUPLICATE_VALUE:Duplicate external id specified: '
 '3482.0:nanoHUB_user_ID__c --","Well, let\'s see what happens when I type '
 'another message here... will the

In [23]:
## second send for dups_df

In [24]:
# create DB2 to Salesforce API object
db_2 = salesforce

Obtained Salesforce access token ...... True


In [32]:
db_2.object_id = object_id
db_2.external_id = external_id

# send data to Salesforce
db_2.send_data(dups_df)

[Success] Bulk job creation successful. Job ID = 7505w00000VryIHAAZ
hello
[Success] CSV upload successful. Job ID = 7505w00000VryIHAAZ
[Success] Closing job successful. Job ID = 7505w00000VryIHAAZ


In [39]:
# check status
db_2.check_bulk_status()

{'id': '7505w00000VryIHAAZ',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-05-07T04:44:08.000+0000',
 'systemModstamp': '2021-05-07T04:44:19.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 3,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 596,
 'apiActiveProcessingTime': 532,
 'apexProcessingTime': 420}

In [40]:
# check status
pprint(db_2.check_bulk_failed_results())

'"sf__Id","sf__Error",NH_survey_results__c,nanoHUB_user_ID__c\n'
